In [1]:
import math
import pandas as pd
from langdetect import detect
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
import string
from sklearn.feature_extraction.text import CountVectorizer
import math
import matplotlib.pyplot as plt

In [2]:
lem = WordNetLemmatizer() #create lemmatizer

In [3]:
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/shirinharandi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
dictionary = pd.read_csv('../data/processedDict.csv')
dictionary['word'] = dictionary['word'].apply(lambda x: lem.lemmatize(x, pos='n'))

In [5]:
# DELETE LATER
# mask = (reviews['date'] >= '2013-01-01') & (reviews['date'] < '2014-01-01')
# reviews = reviews.loc[mask].copy()
# reviews[:1000]

In [6]:
def get_trends_nice_og(reviews):
    
    def get_unique_words(rev):
        
        allwords = rev['comments'].tolist()
        allwords = " ".join(allwords)
        unique_words = set(allwords.split(' '))

        unique_words = list(unique_words)
        unique_words = [string for string in unique_words if string != ""]

        ls = []
        for word in unique_words:
            word = ''.join([i for i in word if not i.isdigit()])
            ls += [word]
        unique_words= ls
        unique_words = [string for string in unique_words if string != ""]
        unique_words = list(dict.fromkeys(unique_words))
        return unique_words
    
    def get_masked_reviews(start_year, end_year):
        mask = (reviews['date'] >= start_year) & (reviews['date'] < end_year)
        return reviews.loc[mask].copy()
    
#     def get_reviews_list(rev):
        
    
    old_reviews =  get_masked_reviews('2013-01-01', '2015-01-01')
#     print(old_reviews)
    new_reviews =  get_masked_reviews('2017-01-01', '2019-01-01')

    start_year_words = get_unique_words(old_reviews)
    end_year_words = get_unique_words(new_reviews)
    
    old_reviews_list = old_reviews['comments'].tolist()
    new_reviews_list = new_reviews['comments'].tolist()

#     print(new_reviews_list)
    
    def get_freq(word, rev):
        total = 0.0
        rev = list(rev)
        for r in rev:
            r = r.split()
            total += math.log(1 + r.count(word))
        return total

    column_names = ["word", "old_frequency", "new_frequency"]
    out = pd.DataFrame(columns = column_names)
    
    old_total = 0.0
    for word in start_year_words:
        f = get_freq(word, old_reviews_list)
        old_total += f
        out = out.append({'word': word, 'old_frequency': f, "new_frequency": 0.0}, ignore_index=True)
        
    new_total = 0.0
    for word in end_year_words:
        f = get_freq(word, new_reviews_list)
        new_total += f
        if word in start_year_words:
            ind = out.index[out['word']==word]
            out.loc[ind,"new_frequency"] = f
        else:
            out = out.append({'word': word, 'old_frequency': 0.0, 'new_frequency': f}, ignore_index=True)
    
    out["old_sum_count"] = old_total
    out["new_sum_count"] = new_total
    return out

In [7]:
killme = []

In [8]:
def get_trends_nice(reviews):
    
    def get_unique_words(rev):
        
        allwords = rev['comments'].tolist()
        allwords = " ".join(allwords)
        unique_words = set(allwords.split(' '))

        unique_words = list(unique_words)
        unique_words = [string for string in unique_words if string != ""]

        ls = []
        for word in unique_words:
            word = ''.join([i for i in word if not i.isdigit()])
            ls += [word]
        unique_words= ls
        unique_words = [string for string in unique_words if string != ""]
        unique_words = list(dict.fromkeys(unique_words))
        return unique_words
    
    def get_masked_reviews(start_year, end_year):
        mask = (reviews['date'] >= start_year) & (reviews['date'] < end_year)
        return reviews.loc[mask].copy()
    
#     def get_reviews_list(rev):
        
    
    old_reviews =  get_masked_reviews('2013-01-01', '2015-01-01')
#     print(old_reviews)
    new_reviews =  get_masked_reviews('2017-01-01', '2019-01-01')

    start_year_words = get_unique_words(old_reviews)
    end_year_words = get_unique_words(new_reviews)
    
    in_first = set(start_year_words)
    in_second = set(end_year_words)

    in_second_but_not_in_first = in_second - in_first

    total_words_in_reviews = start_year_words + list(in_second_but_not_in_first)
    killme = total_words_in_reviews
    
    old_reviews_list = old_reviews['comments'].tolist()
    new_reviews_list = new_reviews['comments'].tolist()


    def get_freq(word, rev):
        total = 0.0
        rev = list(rev)
        for r in rev:
            r = r.split()
            total += math.log(1 + r.count(word))
        return total

    column_names = ["word", "old_frequency", "new_frequency"]
    out = pd.DataFrame(columns = column_names)
    
    old_total = 0.0
    new_total = 0.0
    for word in total_words_in_reviews:
        f = get_freq(word, old_reviews_list)
        old_total += f
#         print(f)
#         print(word)
        f2 = get_freq(word, new_reviews_list)
        new_total += f2
        out = out.append({'word': word, 'old_frequency': f, "new_frequency": f2}, ignore_index=True)
        
    out["old_sum_count"] = old_total
    out["new_sum_count"] = new_total
    return out

In [9]:
dictionary_words = dictionary['word'].tolist()
dictionary_words = list(set(dictionary_words))
# dictionary_words

In [10]:
cities = ["tokyo", "belize", "santiago", "taipei", "singapore", "beijing", "hong_kong", "mexico_city", "buenos_aires", "rio"]
# cities = ["manchester", "tokyo"]

column_names = ["word", "old_frequency", "new_frequency","old_sum_count", "new_sum_count","city", "Ofrequency", "Nfrequency","freq", "gain", "licia_gain"]
big_out = pd.DataFrame(columns = column_names)

for c in cities: 
    path = "/Users/shirinharandi/Desktop/COMP0031/Data/en_reviews/{}_en.csv".format(c)
    filepath = path
    city_name = c
    reviews = pd.read_csv(filepath)

    table = str.maketrans('', '', string.punctuation) #mapping to strip punctuation in review

    #strip punct of each review -> lemmatise -> output is list of words so join into sentences
    reviews['comments'] = reviews.comments.apply(lambda review: ' '.join(map(str, [lem.lemmatize(word.translate(table), pos='n') for word in review.lower().split()])))
    reviews['date'] = pd.to_datetime(reviews['date'])
    
    out = get_trends_nice_og(reviews)
    
    test = out[out['word'].isin(dictionary_words)]

    test['city'] = city_name
    test['Ofrequency'] = test['old_frequency'] / test['old_sum_count']
    test['Nfrequency'] = test['new_frequency'] / test['new_sum_count']
    test['freq'] = (test['Nfrequency'] + test['Ofrequency'])/2
    test['gain'] = np.where(test['Ofrequency'] == 0, 1000 ,test['Nfrequency'] / test['Ofrequency'])
    test['licia_gain'] = np.where(test['Ofrequency'] == 0, 1000 ,test['new_frequency'] / test['old_frequency'])
    
#     print(test)
    big_out = pd.concat([big_out, test], ignore_index=True) 


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/

In [11]:
big_out.to_csv(r'~/Downloads/big_out.csv', index = False)
big_out

,word,old_frequency,new_frequency,old_sum_count,new_sum_count,city,Ofrequency,Nfrequency,freq,gain,licia_gain
0,walkable,2.772589,579.418927,84514.056407,2.853027e+06,tokyo,0.000033,0.000203,0.000118,6.190566,208.981203
1,share,32.290235,372.063688,84514.056407,2.853027e+06,tokyo,0.000382,0.000130,0.000256,0.341326,11.522483
2,mother,17.734145,143.547133,84514.056407,2.853027e+06,tokyo,0.000210,0.000050,0.000130,0.239777,8.094393
3,meal,30.210794,478.927265,84514.056407,2.853027e+06,tokyo,0.000357,0.000168,0.000263,0.469603,15.852853
4,min,234.923769,4131.184119,84514.056407,2.853027e+06,tokyo,0.002780,0.001448,0.002114,0.520920,17.585211
...,...,...,...,...,...,...,...,...,...,...,...
3190,transport,117.495223,224.409787,462327.771895,7.815908e+05,rio,0.000254,0.000287,0.000271,1.129775,1.909948
3191,property,98.482401,196.124344,462327.771895,7.815908e+05,rio,0.000213,0.000251,0.000232,1.177995,1.991466
3192,question,523.523122,871.247441,462327.771895,7.815908e+05,rio,0.001132,0.001115,0.001124,0.984410,1.664200
3193,hot,236.760745,379.309944,462327.771895,7.815908e+05,rio,0.000512,0.000485,0.000499,0.947665,1.602081


In [12]:
column_names = ["word", "gain_mean"]
mean_gain = pd.DataFrame(columns = column_names)

for dic in dictionary_words:
    dic_rows = big_out[big_out['word'] == dic]
    dic_rows = dic_rows[dic_rows['gain'] != 1000]
    f = dic_rows['gain'].mean()
    mean_gain = mean_gain.append({'word': dic, 'gain_mean': f}, ignore_index=True)

mean_gain

,word,gain_mean
0,connected,1.470889
1,walkable,3.921430
2,appartment,0.952284
3,wifi,1.011630
4,helpful,0.852944
...,...,...
316,sweet,0.934639
317,timely,2.508729
318,responded,1.780466
319,cosy,0.970206


In [13]:
top = mean_gain.nlargest(20, 'gain_mean')
bottom = mean_gain.nsmallest(20, 'gain_mean')
top = top.sort_values(by = 'gain_mean', ascending=True)
li = pd.concat([bottom, top], ignore_index=True)
li

,word,gain_mean
0,punctuality,0.153395
1,email,0.221603
2,generosity,0.339193
3,discrete,0.413901
4,arrival,0.476872
5,punctual,0.492361
6,delightful,0.520771
7,waiting,0.523889
8,interesting,0.539752
9,hers,0.545081


In [14]:
main_rows = pd.DataFrame()
for w in li['word']:
#     print(w)
    temp = big_out[big_out['word'] == w]
    temp = temp[temp['gain'] != 1000]
    main_rows = pd.concat([main_rows, temp], ignore_index=True) 

main_rows

,word,old_frequency,new_frequency,old_sum_count,new_sum_count,city,Ofrequency,Nfrequency,freq,gain,licia_gain
0,punctuality,1.386294,1.386294,426329.038844,1.389642e+06,buenos_aires,0.000003,9.975910e-07,2.124646e-06,0.306791,1.000000
1,punctuality,0.693147,0.000000,462327.771895,7.815908e+05,rio,0.000001,0.000000e+00,7.496275e-07,0.000000,0.000000
2,email,27.843670,280.973725,84514.056407,2.853027e+06,tokyo,0.000329,9.848269e-05,2.139694e-04,0.298925,10.091117
3,email,13.862944,85.728235,43668.014643,7.311018e+05,belize,0.000317,1.172590e-04,2.173606e-04,0.369364,6.183985
4,email,61.807882,64.292789,87073.800648,6.873788e+05,santiago,0.000710,9.353328e-05,4.016833e-04,0.131768,1.040204
...,...,...,...,...,...,...,...,...,...,...,...
365,response,0.693147,200.842783,38248.136582,3.599748e+05,beijing,0.000018,5.579356e-04,2.880290e-04,30.787109,289.754888
366,response,66.542129,1308.217847,275248.116448,1.793143e+06,hong_kong,0.000242,7.295671e-04,4.856602e-04,3.017817,19.659994
367,response,11.090355,913.581535,183693.466622,3.037567e+06,mexico_city,0.000060,3.007609e-04,1.805676e-04,4.981610,82.376222
368,response,59.610658,351.831086,426329.038844,1.389642e+06,buenos_aires,0.000140,2.531811e-04,1.965021e-04,1.810724,5.902151


In [15]:
main_rows.to_csv(r'~/Downloads/main_rows.csv', index = False)

In [17]:
def get_cat_lev1(word):
    ind = dictionary.index[dictionary['word']==word]
    return dictionary.loc[ind, 'cat_lev1'].tolist()[0]

main_rows['cat'] = main_rows['word'].apply(lambda x: get_cat_lev1(x))
main_rows

,word,old_frequency,new_frequency,old_sum_count,new_sum_count,city,Ofrequency,Nfrequency,freq,gain,licia_gain,cat
0,punctuality,1.386294,1.386294,426329.038844,1.389642e+06,buenos_aires,0.000003,9.975910e-07,2.124646e-06,0.306791,1.000000,business
1,punctuality,0.693147,0.000000,462327.771895,7.815908e+05,rio,0.000001,0.000000e+00,7.496275e-07,0.000000,0.000000,business
2,email,27.843670,280.973725,84514.056407,2.853027e+06,tokyo,0.000329,9.848269e-05,2.139694e-04,0.298925,10.091117,business
3,email,13.862944,85.728235,43668.014643,7.311018e+05,belize,0.000317,1.172590e-04,2.173606e-04,0.369364,6.183985,business
4,email,61.807882,64.292789,87073.800648,6.873788e+05,santiago,0.000710,9.353328e-05,4.016833e-04,0.131768,1.040204,business
...,...,...,...,...,...,...,...,...,...,...,...,...
365,response,0.693147,200.842783,38248.136582,3.599748e+05,beijing,0.000018,5.579356e-04,2.880290e-04,30.787109,289.754888,business
366,response,66.542129,1308.217847,275248.116448,1.793143e+06,hong_kong,0.000242,7.295671e-04,4.856602e-04,3.017817,19.659994,business
367,response,11.090355,913.581535,183693.466622,3.037567e+06,mexico_city,0.000060,3.007609e-04,1.805676e-04,4.981610,82.376222,business
368,response,59.610658,351.831086,426329.038844,1.389642e+06,buenos_aires,0.000140,2.531811e-04,1.965021e-04,1.810724,5.902151,business


In [18]:
main_rows.to_csv(r'~/Downloads/MAIN.csv', index = False)